In [1]:
import cv2
from skimage import color
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
from scipy import ndimage as ni
import itertools
import pandas as pd
import math
from sklearn.cross_validation import cross_val_score
from sklearn import svm

In [2]:
def convolve(img):
    I_x = np.zeros((img.shape[0] - 2, img.shape[1] - 2))
    I_y = np.zeros((img.shape[0] - 2, img.shape[1] - 2))
    h = I_x.shape[0]
    w = I_y.shape[1]
    s_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    s_y = np.array([[-1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    I_x = ni.convolve(img, s_x, mode='constant', cval=0.0)[1:-1, 1:-1]
    I_y = ni.convolve(img, s_y, mode='constant', cval=0.0)[1:-1, 1:-1]
    return np.sqrt(I_x ** 2 + I_y ** 2), np.abs(np.arctan2(I_y, I_x))

In [3]:
def histograms(img, cellRowsCnt=9, cellColsCnt=9, binCount=8, blockRowCells=4, blockColCells=4, step_x=2, step_y=2, eps=1e-10): 
    G , theta = convolve(img)
    #Итоговые фичи олжны быть одинакового размера
    '''
    (h, w) = theta.shape
    cellRows = np.floor(h / cellRowsCnt)
    cellCols = np.floor(w / cellColsCnt)
    img_hists = np.zeros((cellRowsCnt, cellColsCnt, binCount))
    for i in range(h - h % cellRowsCnt):
        for j in range(w - w % cellColsCnt):
            a = theta[i][j] * binCount / np.pi
            img_hists[i / cellRows, j / cellCols, a] += G[i][j]
    '''     
    hn = 9
    hm = 9
    cellRows = int(theta.shape[0] / hn)
    cellCols = int(theta.shape[1] / hm)
    H = np.zeros((hn, hm, binCount))
    bucket_angle = (math.pi + eps) / binCount
    buckets = np.floor(theta / bucket_angle)
    
    absGPerBucket = []
    for j in range(binCount):
        absGPerBucket.append(G * (buckets == j))
    
    for i in range(hn):
        for j in range(hm):
            for buck in range(binCount):
                H[i][j][buck] = np.sum(absGPerBucket[buck][i * cellRows : (i + 1) * cellRows, j * cellCols : (j + 1) * cellCols])
    return H
    

In [4]:
def descriptor(img_hists, blockRowCells=5, blockColCells=5, step_x=3, step_y=3, eps=1e-15):
    res = []
    for i in range(0, img_hists.shape[0], step_x):
        for j in range(0, img_hists.shape[1], step_y):
            v = np.concatenate(img_hists[i : i + blockRowCells, j : j + blockColCells, :])
            res.append(v / np.sqrt(np.sum(v ** 2) + eps))
    return np.concatenate(np.concatenate(res))

In [5]:
def extract_hog(img, cellRowsCnt=10, cellColsCnt=10, binCount=8, blockRowCells=4, blockColCells=4, step_x=2, step_y=2, eps=1e-10): 
    img_hists = histograms(img)
    return descriptor(img_hists)    

In [6]:
def f(x):
    res = str(x)
    return 'train/' + '0' * (5 - len(res)) + res + '.png'
filenames = [f(x) for x in range(39209)]

In [7]:
images = [color.rgb2gray(io.imread(x)) for x in filenames]

In [8]:
%%time
X = []
for j in tqdm(range(len(images))):
    X.append(extract_hog(images[j]))

100%|██████████| 39209/39209 [02:20<00:00, 293.27it/s]

CPU times: user 2min 20s, sys: 171 ms, total: 2min 20s
Wall time: 2min 20s


In [9]:
y = pd.DataFrame(pd.read_csv('train/gt.csv')).class_id

In [10]:
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
X_train[0].shape

(1352,)

In [ ]:
%%time
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
%%time
clf = svm.SVC(kernel='rbf')
scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')
print(scores)